In [22]:
import csv
from collections import deque
from tabulate import tabulate

# read in from csv file
def parse_file(file_name):
    with open(file_name, 'r') as file:
        # read lines from csv file
        lines = list(csv.reader(file))

        # header parsing
        machine_name = lines[0][0]
        start_state = lines[4][0]
        accept_state = lines[5][0]
        reject_state = lines[6][0]

        # parse transitions
        transitions = {}
        for line in lines[7:]:

            # split lines by commas
            if len(line) == 1:
                line = line[0].split(',')

            # ignore lines with not enough elements
            if not line or len(line) < 5:
                continue

            # split line into parts
            state, read_char, new_state, write_char, direction = line
            # use current state and read character as key
            key = (state, read_char)
            if key not in transitions:
                # initialize list
                transitions[key] = []
            # add the transition
            transitions[key].append((new_state, write_char, direction))
    return machine_name, start_state, accept_state, reject_state, transitions

# simulate ntm with bfs
def simulate_ntm(input_string, transitions, start_state, accept_state, reject_state, max_depth=100):
    # initial configuration (left tape, current state, right tape, depth)
    initial_config = ("", start_state, input_string, 0)
    queue = deque([initial_config])
    # reconstruction path of computation
    parent_map = {initial_config: []}
    configurations_per_level = [1]
    total_transitions = 0
    # flag if all configurations reject
    all_configs_rejected = True
    max_transitions = 1000

    # bfs loop
    while queue and total_transitions < max_depth:
        current_level_size = len(queue)
        next_level_configs = 0

        for _ in range(current_level_size):
            left, state, right, depth = queue.popleft()

            # check for accept and reject states
            if state == accept_state:
                # calculate nondeterminism
                all_configs_rejected = False
                avg_nondeterminism = sum(configurations_per_level) / len(configurations_per_level)
                return "accept", trace_path(parent_map), total_transitions, avg_nondeterminism, depth

            # determine current head character
            head_char = right[0] if right else '_'
            key = (state, head_char)

            # explore possible transitions
            if key in transitions:
                for new_state, write_char, direction in transitions[key]:
                    # update tape and create new configuration
                    new_left, new_right = move_tape(left, right, write_char, direction)
                    new_config = (new_left, new_state, new_right, depth + 1)

                    # avoid same configuration
                    if new_config not in parent_map:
                        parent_map[new_config] = parent_map[(left, state, right, depth)] + [(left, state, right)]
                        queue.append(new_config)
                        next_level_configs += 1

            # increment transition counter
            total_transitions += 1

            if total_transitions >= max_transitions:
                return "stopped", [], total_transitions, 0

        # update nondeterminism
        configurations_per_level.append(next_level_configs)

        # stop if no more paths to explore
        if not queue:
          break

    avg_nondeterminism = sum(configurations_per_level) / len(configurations_per_level)
    return "reject", trace_path(parent_map), total_transitions, avg_nondeterminism, depth

# move tape and update configuration
def move_tape(left, right, write_char, direction):
    if direction == "R":
      # move right, add write_char to left and remove first char from right
        new_left = left + write_char
        new_right = right[1:] if len(right) > 1 else "_"
    elif direction == "L":
      # move left, remove last char from left,
        new_left = left[:-1] if left else "_"
        new_right = (left[-1] if left else "_") + write_char + right
    else:
        raise ValueError("Invalid move direction")
    return new_left, new_right

# trace paths back from parent_map
def trace_path(parent_map):
    paths = []
    for config, path in parent_map.items():
        # add final configuration to path
        full_path = path + [config]
        paths.append(full_path)
    # reverse path to start
    return paths

def main():
    # Parse machine file
    machine_details = parse_file("containsb.csv")
    if not machine_details:
        return

    # get machine details
    machine_name, start_state, accept_state, reject_state, transitions = machine_details

    # test input strings
    test_strings = ["a", "b", "bbbbbb", "aaaabaaa", "bbb", ""]
    results = []

    for input_string in test_strings:
        # simulate turing machine
        result, all_paths, transitions_count, nondeterminism, depth = simulate_ntm(
            input_string, transitions, start_state, accept_state, reject_state
        )
        # combine all paths explored
        formatted_paths = "\n\n".join(
            "\n".join(f"{config[0]} | {config[1]} | {config[2]}" for config in path)
            for path in all_paths
        ) if all_paths else "None"

        if result == "stopped":
          execution_summary = f"Execution stopped after \n{transitions_count} transitions \n(step limit exceeded)."
        elif result == "accept" and all_paths:
            execution_summary = f"String accepted in \n{depth} transitions \non the accepting path."
        else:
            execution_summary = f"String rejected in \n{depth} transitions \n(max depth explored)."

        comment = f"string chosen to \ndisplay different \ndegree of \nnon-determinism and \nas a sanity \ncheck to ensure \nmachine works"

        # add new results to results list
        results.append([
            machine_name,
            input_string,
            result,
            depth,
            transitions_count,
            f"{nondeterminism:.2f}",
            formatted_paths,
            execution_summary,
            comment
        ])

    # generate the table
    headers = [
    "NTM", "String", "Result", "Depth",
    "Configurations", "Avg Non-Det", "All Paths", "Execution Summary", "Comment"
    ]
    # print the table
    print("\nSimulation Results:\n")
    print(tabulate(results, headers=headers, tablefmt="grid"))
if __name__ == "__main__":
    main()


Simulation Results:

+----------------------+----------+----------+---------+------------------+---------------+-----------------------+------------------------+----------------------+
| NTM                  | String   | Result   |   Depth |   Configurations |   Avg Non-Det | All Paths             | Execution Summary      | Comment              |
+======================+==========+==========+=========+==================+===============+=======================+========================+======================+
| ﻿Contains 'b' Machine | a        | reject   |       2 |                3 |          0.75 | | q1 | a              | String rejected in     | string chosen to     |
|                      |          |          |         |                  |               |                       | 2 transitions          | display different    |
|                      |          |          |         |                  |               |  | q1 | a             | (max depth explored).  | degree of       